-----
# cartoGRAPHs
precalculated large networks , e.g. the Interactome

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

-----

In [1]:
from cartoGRAPHs import * 

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import * 
from func_exportVR import * 

_____
# 1 | HUMAN NETWORK
_____

In [2]:
organism = 'human'

G = load_graph(organism)

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

_____ 
# 2 | FIGURE 2A + supplementary | Importance layout (with diverse node lists highlighted)
_____

In [3]:
DM_cent = load_datamatrix(G,organism, 'importance')
DM_cent_mod = DM_cent.round(5)

### 2D PORTRAIT

In [63]:
n_n = 20
spr = 1.0
md = 0.9 
metr = 'cosine'

In [64]:
%%time 

posG_umap2D = layout_portrait_umap(G,DM_cent_mod, 2,
                                   n_neighbors = n_n,
                                   spread = spr,
                                   min_dist = md,
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 4min 59s, sys: 34 s, total: 5min 33s
Wall time: 3min 23s


## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [65]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
#size_nx = list(draw_node_degree(G, scale_factor).values())
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

#### ESSENTIALITY color settings 

In [187]:
color_method = 'essentiality'

l_genes = essential_genes
    
node_col = '#00abff' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, essential_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

#### DEVELOPMENTAL color settings 

In [178]:
color_method = 'oocyte'

if color_method == 'oocyte' or color_method == '2cell' or color_method == '4cell' or color_method == '8cell':

    node_col = '#0181A3' 
    devgenes_selection = color_method
    df_devgenes = pd.read_csv('input/EmExplorer/'+devgenes_selection+'_Homo-sapiens.txt', delimiter = "\t")
    
    devgenes = list(df_devgenes['stage'])

    d_devgenes_entrezid = {}
    for i,k in d_gene_sym.items():
        if k in devgenes:
             d_devgenes_entrezid[k] = i 
    devgenes_entrezid = list(d_devgenes_entrezid.values())
    
    l_genes = devgenes_entrezid
    
    d_col_all = color_nodes_from_list(G, l_genes, node_col)
    colors = list(d_col_all.values())

    edge_color = node_col
    d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
    width_edges = 0.25
    opacity_edges = 0.5
        
else:
    pass

#### CANCER color settings 

In [181]:
disease_category = 'cancer'

color_method = disease_category
disease_genes = get_disease_genes(G, d_names_do, d_do_genes, disease_category)
l_genes = disease_genes

node_col = '#00C9FF' 
d_col_all = color_nodes_from_list(G, disease_genes, node_col)
colors = list(d_col_all.values())

edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)

width_edges = 0.25
opacity_edges = 0.25

#### RARE DISEASE genes color settings 

In [76]:
import xml.etree.ElementTree as ET
tree = ET.parse('input/orphadata.xml')
root = tree.getroot()

# retrieve genes from all rare diseases from database/file  
disorder_to_genes = {}
for disorder in root.iter('Disorder'):
    dis = list(disorder.attrib.values())[0]
    for elem in disorder.findall('DisorderGeneAssociationList'):
        genes = []
        for disorderassoc in elem.findall('DisorderGeneAssociation'):
            for gene in disorderassoc.findall('Gene'):
                for reference in gene.findall('ExternalReferenceList'):
                     for refs in reference.findall('ExternalReference'):
                        for ensg in refs.findall('Source'):
                            if ensg.text == 'Ensembl':
                                for e in refs.findall('Reference'):
                                    ensgID = (e.text)
                            else:
                                pass
                            
            genes.append(ensgID)
    disorder_to_genes[dis] = genes

    
# get a list of all rare disease genes obtained 
all_raredisease_genes = []
for dis,genes in disorder_to_genes.items():
    for i in genes:
        all_raredisease_genes.append(i)
      
    
# make sure there are no duplicates
set_all_raredisease_genes = set(all_raredisease_genes)


# convert ensgID to entrezID and get gene symbols
ensg_to_entrez = pd.read_csv('input/ensg_to_entrezid.txt',sep="	")

ensg = list(ensg_to_entrez['From'])
entrez = [str(int(i)) for i in ensg_to_entrez['To'][:-1]] #list(ensg_to_entrez['To'])
d_ensg_entrez = dict(zip(ensg,entrez))


# match ensgID with entrezID
set_all_raredisease_genes_id = {}
for ix,entrez in d_ensg_entrez.items():
    if ix in set_all_raredisease_genes:
        set_all_raredisease_genes_id[ix]=entrez

        
# color all rare disease genes  
l_rarediseasegenes = list(set_all_raredisease_genes_id.values())
color_method = 'rarediseasegenes'

l_genes = l_rarediseasegenes

node_col =  '#004EAB'#'#0A81A1' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

## PLOT NETWORK PORTRAIT - plotly 

In [77]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [81]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, 
    umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutImportance_'+color_method+'_'+organism

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_NetlayoutImportance_rarediseasegenes_human_onlynodes.html'

## PLOT NETWORK PORTRAIT - networkx

In [ ]:
# Choose edges to highlight that link inbetween 2 essential genes only 
d_edge_col = color_edges_from_nodelist_specific(G, essential_genes, node_col)

# Rest edges 
d_rest_edges={}
for e in G.edges():
    if e not in d_edge_col.keys():
        d_rest_edges[e] = '#f0f0f0'

d_all_edges = {**d_edge_col, **d_rest_edges}
d_all_edges_sort = {key:d_all_edges[key] for key in G.edges()}
edge_color = list(d_all_edges_sort.values())



plt.figure(figsize=(25,25))
plt.title('2D Portrait', size=16)

nx.draw_networkx_nodes(G, 
                       posG,
                       edgecolors = 'dimgrey', 
                       linewidths = 1.2, 
                       node_color = colors, 
                       node_size = size_nx, 
                       alpha = 0.8)

nx.draw_networkx_edges(G, posG, width = 0.1, 
                       edge_color = edge_color, 
                      alpha = 0.25)

plt.box(False)

fname = 'output_plots/2Dlayouts/2Dportrait_Fig2a_'+organism

# PNG
#plt.savefig(fname+'.png',dpi=300)

# PDF 
plt.savefig(fname+'.pdf',format='pdf')

plt.show()

_____ 
# 3 | FIGURE 2B - disease layout
_____

## DISEASE LAYOUT

In [68]:
DM_global = load_datamatrix(G,organism, 'global')

In [107]:
df_max = DM_global.max()
l_max_visprob = max(list(df_max.values))

0.906243

In [161]:
enhance_factor = 0.95 #l_max_visprob  # for emphasizing functional features > should be >= max of structural matrix values  # if significantly higher than max values in structural matrix > causes isolation of nodes with functional features enhanced
print('ENHANCING FACTOR functional features: ',enhance_factor)
n_n = 12
spr = 1
md = 0.0
metr = 'cosine'

ENHANCING FACTOR functional features:  0.95


## 1. | NF1 + neighbors 

In [162]:
path_nf = 'input/neurofibromatosis/'
#G_neuro = nx.read_edgelist(path_nf+'nf_edges.txt', delimiter=',')

# gene of interest
nf_goi_ = str(int(np.loadtxt(path_nf+'nf_gene_of_interest.txt', unpack=False)))
nf_goi = list(nf_goi_.split(" "))
nf_goi_str = ' '.join([str(item) for item in nf_goi])

# pathway genes
#nf_path_ = np.loadtxt(path_nf+'nf_pathway_genes.txt', unpack=False)
#nf_pathw = []
#for i in nf_path_:
#    nf_pathw.append(str(int(i)))
    
# NF1 neighbors 
#df_nf_neigh = pd.read_csv(path_nf+'nf_neighbors.txt', header = None,index_col=False)
#nf_neigh = [str(i) for i in list(df_nf_neigh[0])]
#len(nf_neigh)

nf_neighbors = []
for edge in G.edges():
    if edge[0] == nf_goi_str:
        nf_neighbors.append(edge[1])
    elif edge[1] == nf_goi_str: 
        nf_neighbors.append(edge[0])

enhance_genes = nf_goi + nf_neighbors # + nf_pathw + nf_neigh

### Layout for NF1 + Neighbors

In [163]:
d_vec_disease = {}
for i in G.nodes():
    if i in enhance_genes:
        d_vec_disease[i]=enhance_factor
    else:
        d_vec_disease[i]=0
        
d_vec_disease_sorted = {key:d_vec_disease[key] for key in G.nodes()}

DF_nf = pd.DataFrame(list(d_vec_disease_sorted.values()))
DF_nf.index = list(G.nodes())
DF_nf.columns = ['NF1']

DF_nf = DF_nf.T
DF_structural = DM_global 
DF_structural.index = DM_global.columns

DF_merge_nf = pd.concat([DF_structural, 
                      DF_nf]).T

In [164]:
DF_merge_nf

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,8001,51351,1551,51458,143903,10861,51471,221044,29965,NF1
66008,0.900242,0.002519,0.002502,0.002502,0.002525,0.002517,0.002512,0.002511,0.002502,0.002501,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.000000,0.0
8473,0.000687,0.900245,0.000002,0.000002,0.000685,0.000008,0.000031,0.000006,0.000002,0.000002,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.000000,0.0
2561,0.012865,0.000036,0.900367,0.000036,0.000036,0.000065,0.000037,0.000036,0.000036,0.000036,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.000000,0.0
3759,0.006433,0.000018,0.000018,0.900389,0.000018,0.000027,0.000041,0.000018,0.000018,0.000018,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.000000,0.0
22906,0.005347,0.005316,0.000015,0.000015,0.900120,0.005314,0.000026,0.000015,0.000015,0.000016,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.00000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.000000,0.000000,0.000001,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,...,0.0,0.00007,0.0,0.0,0.90007,0.00007,0.000000,0.00000,0.000000,0.0
10861,0.000000,0.000000,0.000001,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0.000000,...,0.0,0.00007,0.0,0.0,0.00007,0.90007,0.000000,0.00000,0.000000,0.0
51471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000054,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.900052,0.00000,0.000000,0.0
221044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000003,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.90003,0.000000,0.0


### 2D PORTRAIT

In [165]:
%%time 

posG_umap2D = layout_portrait_umap(G,DF_merge_nf, 2,
                                   n_neighbors = n_n,
                                   spread = spr,
                                   min_dist = md,
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 2min 13s, sys: 8.56 s, total: 2min 21s
Wall time: 1min 24s


## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [166]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

#### NF1 SPECIFIC 

In [167]:
color_method = 'NF1'
l_genes = enhance_genes

node_col = '#00E3DF' #'#FF8E00' 
undefined_col = '#DEDEDE'
rest_col_nodes = '#DEDEDE'

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_node(G, nf_goi, node_col)

width_edges = 0.7
opacity_edges = 0.7

### PLOT NETWORK PORTRAIT - plotly 

In [168]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [169]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutDisease_NF1Neighbors_enhance'+str(enhance_factor)+'_'+color_method+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_NetlayoutDisease_NF1Neighbors_enhance0.95_NF1_nn12_spr1_md0.0_cosine_human.html'

## 2. | NF1 in Disease Module
e.g.
+ DOID:0111253 / Neurofibromatosis1
+ DOID:962 / Neurofibroma
+ DOID:0050736 / Autosomal dominant disease
+ DOID:9001734 / Neurocutaneous Syndromes

In [171]:
# get Disease Feature Matrix 

FM_disease = pd.read_csv('input/Features_Disease_Dataframe_'+organism+'.csv', index_col=0)

num_doid = '0050736'

# define specific DOID 
dismod = FM_disease.loc[:,'DOID:'+num_doid]
dismod_id = 'DOID'+num_doid

dismod_genes = [] 
for i,v in enumerate(dismod.values):
    if v == 1:
        dismod_genes.append(dismod.index[i])
    else:
        pass
    
l_genes = dismod_genes
len(l_genes)

398

In [172]:
# create an empty matrix for zeros of rest genes (not associated to any disease)
rest = []
for i in G.nodes():
    if str(i) in dismod.index or int(i) in dismod.index:
        pass
    else: 
        rest.append(i)

empty = np.zeros(shape=(len(rest),1))
empty_stacked = np.vstack(empty)
df_empty = pd.DataFrame(empty_stacked)
df_empty.index = rest

# combine disease vector with rest of genes in graph
# non-associated genes = 0 
# specific DOID associated genes = 1 
df_dismod_complete = pd.concat([dismod, df_empty])

# reindex the dataframe to match the index of the structural dataframe 
order_structural = list(DF_structural.index)
df_dismod_complete.index = order_structural
df_dismod_complete.columns = ['Disease Module']

DF_dismod = df_dismod_complete * enhance_factor 
DF_dismod.index = DM_global.columns

DF_merge_dismod = pd.concat([DF_structural, DF_nf.T,
                      DF_dismod], axis=1)

DF_merge_dismod

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,51351,1551,51458,143903,10861,51471,221044,29965,NF1,Disease Module
66008,0.900242,0.000687,0.012865,0.006433,0.005347,0.000460,0.000051,0.000345,0.003464,0.002905,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0
8473,0.002519,0.900245,0.000036,0.000018,0.005316,0.000005,0.000002,0.000003,0.000012,0.000008,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0
2561,0.002502,0.000002,0.900367,0.000018,0.000015,0.000002,0.000000,0.000001,0.000010,0.000008,...,0.000001,0.0,0.0,0.000001,0.000001,0.000000,0.00000,0.000000,0.0,0.0
3759,0.002502,0.000002,0.000036,0.900389,0.000015,0.000002,0.000000,0.000001,0.000010,0.000008,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0
22906,0.002525,0.000685,0.000036,0.000018,0.900120,0.000459,0.000000,0.000001,0.000010,0.000009,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000070,0.0,0.0,0.900070,0.000070,0.000000,0.00000,0.000000,0.0,0.0
10861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000070,0.0,0.0,0.000070,0.900070,0.000000,0.00000,0.000000,0.0,0.0
51471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.900052,0.00000,0.000000,0.0,0.0
221044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.90003,0.000000,0.0,0.0


In [173]:
%%time 

posG_umap2D = layout_portrait_umap(G,DF_merge_dismod, 2, 
                                   n_neighbors = n_n, 
                                   spread = spr, 
                                   min_dist = md, 
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 2min 11s, sys: 10.7 s, total: 2min 22s
Wall time: 1min 29s


## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [174]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
#size_nx = list(draw_node_degree(G, scale_factor).values())
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

In [175]:
color_method = 'NF1module'
l_genes = enhance_genes

node_col = '#00E3DF' #'#FF8E00' 
undefined_col = '#DEDEDE'
rest_col_nodes = '#DEDEDE'

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)

width_edges = 0.55
opacity_edges = 0.55

## PLOT NETWORK PORTRAIT - plotly 

In [176]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [177]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutDisease_NF1module_'+dismod_id+'_enhance'+str(enhance_factor)+'_'+color_method+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_NetlayoutDisease_NF1module_DOID0050736_enhance0.95_NF1module_nn12_spr1_md0.0_cosine_human.html'

## 3. | NF1 in Disease Context 
which ones ? 

### 2D PORTRAIT

In [102]:
n_n = 8
spr = 1
md = 0.0
metr = 'cosine'

In [103]:
%%time 

posG_umap2D = layout_portrait_umap(G,DF_merge, 2,
                                   n_neighbors = n_n,
                                   spread = spr,
                                   min_dist = md,
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 1min 32s, sys: 8.58 s, total: 1min 41s
Wall time: 1min 3s


## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [107]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
#size_nx = list(draw_node_degree(G, scale_factor).values())
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

In [108]:
color_method = 'NF1'
l_genes = enhance_genes

node_col = '#00E3DF' #'#FF8E00' 
undefined_col = '#DEDEDE'
rest_col_nodes = '#DEDEDE'

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.5

## PLOT NETWORK PORTRAIT - plotly 

In [109]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [110]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutDisease_NF1DISEASECONTEXT_enhance'+str(enhance_factor)+'_'+color_method+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_NetlayoutDisease_enhance0.3_NF1_nn8_spr1_md0.0_cosine_human.html'

# TO DO : layout TRIAL WITH COMPLETE FEATURE MATRIX 

### TOPOGRAPHIC MAP 

In [58]:
# ---------------------------------------------------------------------------
#
# Choose a z-parameter e.g. essentiality, centravg, disease 
# or choose any dictionary with nodeID: z-value
# by setting d_z = {nodeID: val, ...}
#
# ---------------------------------------------------------------------------
z_feat = 'disease'
# ---------------------------------------------------------------------------

if z_feat == 'essentiality':

    ''' Essentiality state of each node displayed on layers of "essential", "non-essential", "non-categorized" ''' 

    value_ess = 5
    value_noness = 3 
    value_notdef = 1

    d_ess = {}
    for i in essential_genes:
        d_ess[i] = value_ess
    d_noness = {}
    for i in non_ess_genes:
        d_noness[i] = value_noness
    d_notdef = {}
    for i in notdefined_genes:
        d_notdef[i] = value_notdef

    d_alless_unsort = {**d_ess,**d_noness,**d_notdef}
    d_z = {key:d_alless_unsort[key] for key in G.nodes()}
    
    l_genes = essential_genes
    
    node_col = '#00abff' 
    undefined_col = '#d3d3d3'
    rest_col_nodes = '#d3d3d3'

    d_col_all = color_nodes_from_list(G, essential_genes, node_col)
    colors = list(d_col_all.values())
    
    edge_color = '#ACACAC' 
    d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
    width_edges = 0.15
    opacity_edges = 0.1
    
    umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size, opacity_nodes)
    umap_edges = get_trace_edges_specific3D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)  
    data = [umap_edges, umap_nodes]
    
    
elif z_feat == 'centravg':
    
    ''' Average of Centrality Metrics displayed in z-height ''' 
    
    d_clos = {}
    for k,v in d_centralities.items():
        d_clos[k] = v[1]

    d_nodecolors = d_clos 
    col_pal = 'YlOrRd'

    d_colors = color_nodes_from_dict(G, d_nodecolors, palette = col_pal)
    colors = list(d_colors.values())
    
    edge_color = '#ACACAC' 
    width_edges = 0.15
    opacity_edges = 0.1
    
    umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size, opacity_nodes)
    umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.05, linewidth = 0.5)
    data = [umap_edges, umap_nodes]
    
    
elif z_feat == 'disease':
    
    ''' Disease count associated to a node displayed in z-height ''' 

    DM_feature  = pd.read_csv('input/Features_Disease_Dataframe_Human.csv', index_col=0)

    # set gene list (= G.nodes())
    genes_assoc = []
    for i in DM_feature.index:
        genes_assoc.append(str(i))

    genes_notassoc = [] 
    for g in G.nodes():
        if g not in genes_assoc:
            genes_notassoc.append(g)

    features_counted = (DM_feature == 1).astype(int).sum(axis=1)
    d_features_counted = dict(zip(genes_assoc, list(features_counted)))
    
    d_rest = {}
    for i in genes_notassoc: 
        d_rest[i] = -20

    d_param_unsorted = {**d_features_counted, **d_rest}
    d_z = {key:d_param_unsorted[key] for key in G.nodes()}
    
    d_clos = {}
    for k,v in d_centralities.items():
        d_clos[k] = v[1]

    d_nodecolors = d_clos 
    col_pal = 'YlOrRd'

    d_colors = color_nodes_from_dict(G, d_nodecolors, palette = col_pal)
    colors = list(d_colors.values())
    edge_color = '#ACACAC'
    width_edges = 0.15
    opacity_edges = 0.1

    umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size, opacity_nodes)
    umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.05, linewidth = 0.5)
    data = [umap_edges, umap_nodes]
    
else:
    pass

#### PLOT 

In [59]:
# ---------------------------------------------------------------------------
#
# Choose a 2D layout as basis
#
# 1. either calculate one e.g.: 
# posG_2Dportrait = layout_portrait_umap(G,DM,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 
# ---------------------------------------------------------------------------
#
# 2. or use a precalculated 2D layout 
posG_2D = posG_umap2D
# ---------------------------------------------------------------------------

posG = layout_topographic(posG_2D, d_z)

path = 'output_plots/Topographic/'
schema = 'light' #'dark'
fname = 'topographic_z_'+z_feat+'_'+netlayout+'_'+organism+'_'+schema

plot_3D(data,path,fname, schema)

'output_plots/Topographic/topographic_z_disease_local_human_light.html'

### GEODESIC MAP

In [60]:
# ---------------------------------------------------------------------------
#
# Choose a radius-parameter e.g. autocore, essentiality
# or choose any dictionary with nodeID: z-value
# by setting d_radius = {nodeID: val, ...}
#
# ---------------------------------------------------------------------------
r_feat = 'autocore'
# ---------------------------------------------------------------------------


if r_feat == 'autocore':

    # Load Gene Information for Gene lists 
    variants = pd.read_csv('input/julias_variants_mod.txt')
    variant_list = [str(i) for i in list(variants['variants'])]
    variant_sym = pd.read_csv('input/julias_variants_sym.txt')
    genes_sym_list = [str(i) for i in list(variant_sym['variant_sym'])]
    d_gene_sym_variants = dict(zip(variant_list,genes_sym_list))

    df_seeds = pd.read_csv('input/seeds_from_genelists.txt')
    df_seeds.columns = ['seeds']
    df_seeds_z = pd.read_csv('input/seeds_from_zNumbers.txt', sep='\n')
    seeds_list = [str(i) for i in list(df_seeds['seeds'])]

    d_seeds_idsym = {}
    for k,v in d_gene_sym.items():
        for i in seeds_list:
            if v == i:
                d_seeds_idsym[k]=i
    seed_list = list(d_seeds_idsym.keys())

    FM_BP = pd.read_csv('input/Features_GO_BiolProc_Dataframe_human.csv', index_col=0)
    bioproc_list = list(FM_BP.index)

    # From obtained Gene lists select those to be mapped onto different shells 
    # select genes for shells 
    shell_one = variant_list
    shell_two = seed_list
    shell_three = [str(i) for i in bioproc_list]

    small_rad = 1
    mid_rad = 5
    large_rad = 30
    outershell_rad = 50

    d_param = {}
    for i in G.nodes():
        if str(i) in variant_list:
            d_param[str(i)] = small_rad
        elif i in seed_list:
            d_param[str(i)] = mid_rad 
        elif int(i) in bioproc_list:
            d_param[str(i)] = large_rad 
        else:
            d_param[str(i)] = outershell_rad

    d_radius = d_param
    genes_rad = variant_list + seed_list + bioproc_list

    # check how many shells of spheres there will be :
    print('Number of Spherical Shells:', len(set(d_param.values())))


    # Specifying coloring based on genes on different shells 
    d_nodecol = d_clos 
    d_colors = color_nodes_from_dict(G, d_nodecol, palette = col_pal)

    d_colors_spec_genes = {}
    for k,v in d_colors.items():
        if k in shell_one:
            d_colors_spec_genes[k]='#8b0000' 
        elif k in shell_two:
            d_colors_spec_genes[k]='#FF4500' 
        elif k in shell_three:
            d_colors_spec_genes[k]=v
        else: 
            d_colors_spec_genes[k]='#d3d3d3'

    d_colors_spec_genes_sort = {key:d_colors_spec_genes[key] for key in G.nodes()}
    colors = list(d_colors_spec_genes_sort.values())

    
elif r_feat == 'essentiality':

    small_rad = 1
    mid_rad = 10
    outershell_rad = 20

    d_ess_scores = {}
    for i in G.nodes():
        if i in essential_genes:
            d_ess_scores[i] = small_rad
        elif i in non_ess_genes:
            d_ess_scores[i] = mid_rad
        elif str(i) in notdefined_genes:
            d_ess_scores[i] = outershell_rad

    d_radius = d_ess_scores
    genes_rad = list(G.nodes())

    # check how many shells of spheres there will be :
    print('Number of Spherical Shells:', len(set(d_param.values())))

    # Specifying coloring based on genes on different shells 
    d_colors_spec_genes = {}
    for k in G.nodes():
        if k in essential_genes:
            d_colors_spec_genes[k]='#0080bf' 
        elif k in non_ess_genes:
            d_colors_spec_genes[k]='#95dcff' 
        else: 
            d_colors_spec_genes[k]='#d3d3d3'

    d_colors_spec_genes_sort = {key:d_colors_spec_genes[key] for key in G.nodes()}
    colors = list(d_colors_spec_genes_sort.values())
    
    
else: 
    pass

Number of Spherical Shells: 4


#### PLOT 

In [65]:
# ---------------------------------------------------------------------------
#
# SELECT a r - Parameter:
#
# d_radius > a dictionary with keys=G.nodes and values=any radius assigned to each node
# ---------------------------------------------------------------------------

posG_sphere = layout_geodesic(G, d_radius, n_neighbors=20, spread=1, min_dist=0.0)#, DM=DM)
posG = posG_sphere

umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size3d, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.05, linewidth = 0.5)
data = [umap_nodes]#,umap_edges]

path = 'output_plots/Geodesic/'
schema = 'light' #'dark'
fname = 'geodesic_r_'+r_feat+'_'+netlayout+'_'+organism+'_'+schema

plot_3D(data,path,fname, 'dark')

DM precalc used


'output_plots/Geodesic/geodesic_r_r-autocore_local_human_light.html'